In [2]:
import os
import fitz
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone
from tqdm.notebook import tqdm

# Carregar variáveis
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
INDEX_NAME = "brsupply-sac-agent"
NAMESPACE = "documentos-informativos"


c:\Users\arthu\OneDrive\Área de Trabalho\BrSupplAI-SAC\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join([page.get_text() for page in doc])

def chunk_text_into_docs(text, source_name, chunk_size=512, chunk_overlap=80):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(text)
    return [
        Document(page_content=chunk, metadata={"source": source_name})
        for chunk in chunks
    ]


In [9]:
from tqdm import tqdm  # Corrigido

pdf_dir = "data/raw/PDF"
all_docs = []

for filename in tqdm(os.listdir(pdf_dir)):
    if filename.endswith(".pdf"):
        path = os.path.join(pdf_dir, filename)
        print(f"📄 Processando: {filename}")
        raw_text = extract_text_from_pdf(path)
        docs = chunk_text_into_docs(raw_text, source_name=filename)
        all_docs.extend(docs)

print(f"Total de chunks gerados: {len(all_docs)}")


📄 Processando: Criação de Usuários.pdf
📄 Processando: Dados-Cadastrais-BRS-SA.pdf
📄 Processando: Manual - Supply Manager (portal de compras).pdf
📄 Processando: Manual Criação de Conta e Subconta.pdf
📄 Processando: Manual do Usuário.pdf




100%|██████████| 6/6 [00:00<00:00, 15.55it/s]

📄 Processando: Meu Mix - Catálogo Completo.pdf
Total de chunks gerados: 624


In [10]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [13]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
import time

BATCH_SIZE = 10  # mais seguro, evita ultrapassar 4MB por chamada
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = PineconeVectorStore(
    index_name=INDEX_NAME,
    embedding=embedding_model,
    namespace=NAMESPACE
)

for i in tqdm(range(0, len(all_docs), BATCH_SIZE), desc="🔁 Indexando em batches"):
    batch = all_docs[i:i + BATCH_SIZE]
    try:
        vectorstore.add_documents(batch)
    except Exception as e:
        print(f"❌ Erro no batch {i}–{i+BATCH_SIZE}: {e}")
        time.sleep(1)  # opcional: pausa em caso de erro


🔁 Indexando em batches: 100%|██████████| 63/63 [02:08<00:00,  2.04s/it]


In [14]:
import os
import pandas as pd
import joblib

DATA_DIR = "data/raw/CSV"
CACHE_FILE = "data/processed/dataframes_cache.pkl"

def carregar_arquivos_csv():
    dataframes = {}
    for filename in os.listdir(DATA_DIR):
        if filename.endswith(".csv"):
            nome_df = filename.replace(".csv", "").lower()
            df = pd.read_csv(os.path.join(DATA_DIR, filename), encoding="utf-8")
            dataframes[nome_df] = df
    return dataframes

def salvar_cache(dataframes):
    os.makedirs("data/processed", exist_ok=True)
    joblib.dump(dataframes, CACHE_FILE)

def carregar_cache():
    if os.path.exists(CACHE_FILE):
        return joblib.load(CACHE_FILE)
    return None

def obter_dataframes():
    cache = carregar_cache()
    if cache:
        return cache
    dfs = carregar_arquivos_csv()
    salvar_cache(dfs)
    return dfs


In [15]:
def validar_usuario(login_digitado: str, df_usuarios: pd.DataFrame) -> bool:
    logins_validos = df_usuarios["Login"].astype(str).tolist()
    return login_digitado in logins_validos


In [22]:
dfs = obter_dataframes()
df_usuarios = dfs["usuarios"]

login = "246738"
login_formatado = login.strip().zfill(11)

if validar_usuario(login_formatado, df_usuarios):
    print("✅ Acesso autorizado")
else:
    print("❌ Acesso negado. Contate o Gerente de Atendimento ao Cliente, Renan Oliveira, em renan.oliveira@brsupply.com.br")


❌ Acesso negado. Contate o Gerente de Atendimento ao Cliente, Renan Oliveira, em renan.oliveira@brsupply.com.br


In [25]:
from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

retriever = PineconeVectorStore.from_existing_index(
    index_name="brsupply-sac-agent",
    embedding=embeddings,  # ✅ CORRETO
    namespace="documentos-informativos"
).as_retriever()

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


In [30]:
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=dfs["pedidos_detalhados"],
    verbose=True,
    allow_dangerous_code=True
)


In [33]:
agent.run("Quantos pedidos foram feitos em março e qual o valor total?")




> Entering new AgentExecutor chain...
Thought: To find out how many orders were made in March and the total value, I need to filter the dataframe for orders made in March and then sum the 'Valor Total' for those orders. I will first convert the 'Data Pedido' column to a datetime format to facilitate filtering by month.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd

# Convert 'Data Pedido' to datetime
df['Data Pedido'] = pd.to_datetime(df['Data Pedido'], format='%d/%m/%Y %H:%M')

# Filter for orders made in March
march_orders = df[df['Data Pedido'].dt.month == 3]

# Count the number of orders and sum the 'Valor Total'
num_orders_march = march_orders['Pedido'].nunique()
total_value_march = march_orders['Valor Total'].replace({'R$ ': '', '.': '', ',': '.'}, regex=True).astype(float).sum()

num_orders_march, total_value_march
```ValueError: could not convert string to float: ''It seems there is an issue with converting the 'Valor Total' column to a float. This cou

<unknown>:2: SyntaxWarning: invalid escape sequence '\$'
<unknown>:2: SyntaxWarning: invalid escape sequence '\.'


I now know the final answer.

Final Answer: In March, there were 56 orders made with a total value of R$ 170,254.60.

> Finished chain.


'In March, there were 56 orders made with a total value of R$ 170,254.60.'